In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=748281b1c537f6919360b2b496f3124e0ad7771336aa12b2e6ce426363c252c4
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.7MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F20 '''
def F20(X):
    f = bn.F20()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 30.57 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_204000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_20_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(20, 30):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i-20,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 20
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=404563, Sun Apr 19 13:25:09 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.805332340524202e+06 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 1.775319803192510e+06 1.0e+00 2.39e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [2159496.26730251] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [1972343.53386317] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 1.500995202424466e+06 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3
   35    665 2.934149999258656e+05 1.1e+00 1.97e+00  2e+00  2e+00 0:03.3
   77   1463 1.245053063067887e+04 1.2e+00 1.65e+00  2e+00  2e+00 0:07.3
  100   1900 -4.519816998825516e+05 1.2e+00 1.42e+00  1e+00  1e+00 0:09.5
  164   3116 -1.301100047306157e+06 1.2e+00 8.25e-01  8e-01  8e-01 0:15.6
  200   3800 -1.836941563370188e+06 1.2e+00 5.51e-01  5e-01  6e-01 0:19.0
  285   5415 -2.246658881313677e+06 1.2e+00 2.16e-01  2e-01  2e-01 0:27.0
  300   5700 -2.277727311841710e+06 1.2e+00 1.85e-01  2e-01  2e-01 0:28.4
  400   7600 -2.329892997437750e+06 1.2e+00 6.08e-02  6e-02  6e-02 0:37.8
  500   9500 -2.335970635271740e+06 1.2e+00 2.10e-02  2e-02  2e-02 0:47.2
  600  11400 -2.336672783056994e+06 1.2e+00 6.97e-03  7e-03  7e-03 0:56.7
  700  13300 -2.336754299324760e+06 1.2e+00 2.33e-03  2e-03  2e-03 1:06.1
  800  15200 -2.336765430841228e+06 1.2e+00 7.46e-04  7e-04  7e-04 1:15.5
  900  17100 -2.336766344181959e+06 1.3e+

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [1969341.12838215] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [2505402.51431526] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     38 1.666895903985188e+06 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2
    3     57 1.531159004003491e+06 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3


...

    3     57 1.661830876709759e+06 1.0e+00 2.33e+00  2e+00  2e+00 0:00.3
   37    703 3.334047742002336e+05 1.1e+00 2.01e+00  2e+00  2e+00 0:03.3
   82   1558 -3.138601214864156e+05 1.2e+00 1.51e+00  2e+00  2e+00 0:07.4
  100   1900 -4.532283159085172e+05 1.2e+00 1.29e+00  1e+00  1e+00 0:09.1
  164   3116 -1.475120122035624e+06 1.2e+00 7.27e-01  7e-01  7e-01 0:15.2
  200   3800 -1.872998660633140e+06 1.2e+00 5.02e-01  5e-01  5e-01 0:18.5
  285   5415 -2.260693422778998e+06 1.2e+00 2.01e-01  2e-01  2e-01 0:26.5
  300   5700 -2.275256276792368e+06 1.2e+00 1.69e-01  2e-01  2e-01 0:27.9
  400   7600 -2.329643457416654e+06 1.2e+00 5.93e-02  6e-02  6e-02 0:37.1
  500   9500 -2.336127359654780e+06 1.2e+00 1.95e-02  2e-02  2e-02 0:46.4
  600  11400 -2.336673067214513e+06 1.2e+00 6.67e-03  7e-03  7e-03 0:55.6
  700  13300 -2.336757610719150e+06 1.2e+00 2.31e-03  2e-03  2e-03 1:04.5
  800  15200 -2.336765568947830e+06 1.2e+00 7.54e-04  7e-04  8e-04 1:13.8
  900  17100 -2.336766366118268e+06 1.2e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [1969341.12838215] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [2155891.53529694] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     38 1.989640047731077e+06 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2
    3     57 1.589407306136679e+06 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [2042782.70930842] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   36    684 5.698042605050802e+05 1.1e+00 1.94e+00  2e+00  2e+00 0:03.4
   79   1501 4.433810034525115e+05 1.2e+00 1.70e+00  2e+00  2e+00 0:07.4
  100   1900 3.291003945391811e+05 1.2e+00 1.50e+00  1e+00  2e+00 0:09.4
  165   3135 8.448619662957801e+04 1.2e+00 1.02e+00  1e+00  1e+00 0:15.5
  200   3800 -1.413990032817703e+05 1.2e+00 7.38e-01  7e-01  7e-01 0:18.7
  286   5434 -3.199963448677143e+05 1.3e+00 3.78e-01  4e-01  4e-01 0:26.8
  300   5700 -3.417451155413514e+05 1.3e+00 3.47e-01  3e-01  4e-01 0:28.1
  400   7600 -3.880531383687573e+05 1.3e+00 1.77e-01  2e-01  2e-01 0:37.4
  500   9500 -4.034229136658433e+05 1.3e+00 1.04e-01  1e-01  1e-01 0:46.8
  600  11400 -4.126945900321567e+05 1.4e+00 7.54e-02  7e-02  8e-02 0:56.1
  700  13300 -4.169070733347328e+05 1.6e+00 5.30e-02  5e-02  6e-02 1:05.5
  800  15200 -4.201758882428559e+05 1.9e+00 5.46e-02  5e-02  7e-02 1:14.8
  900  17100 -4.220078919524604e+05 2.1e+00 3.56e-02  3e-02  5e-02 1:24.2
 1000  19000 -4.227543833969735e+05 2.1e+0

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('sample_data\\F20_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F20(X_Values.iloc[i,:200])
    SVM_Fun [i] = F20(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F20(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([208029.50569788]), count=array([1])) 0.0038129691879617822
SVM
ModeResult(mode=array([1708857.73937795]), count=array([1])) 266261.6782198814
ELN
ModeResult(mode=array([71640.12470151]), count=array([1])) 7433.941583146673
